In [ ]:
!rm -rf AtomML-Course
!git clone https://github.com/AMLS-PRG/AtomML-Course

Cloning into 'AtomML-Course'...
remote: Enumerating objects: 2751, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 2751 (delta 87), reused 21 (delta 21), pack-reused 2625 (from 1)
Receiving objects: 100% (2751/2751), 47.27 MiB | 20.98 MiB/s, done.
Resolving deltas: 100% (1356/1356), done.


In [ ]:
pip install ase matplotlib numpy
from ase.io.lammpsrun import read_lammps_dump
from ase.geometry import radial_distribution_function
import numpy as np

In [ ]:
# --------- File paths for liquid and solid MD trajectories ---------
file_liquid = "AtomML-Course/module-5/04-Performing-MD-simulations/runLammps/si.lammps-dump-text"
file_solid  = "AtomML-Course/module-5/04-Performing-MD-simulations/runLammps/si_solid.lammps-dump-text"

# --------- RDF settings ---------
rdf_cutoff = 10.0  # cutoff radius in Ångströms (1 nm)
rdf_bins = 200     # number of bins for RDF histogram

# --------- Read the last 70 frames to represent the equilibrium state ---------
frames_liquid = read_lammps_dump(file_liquid, index=slice(-700, None))
frames_solid  = read_lammps_dump(file_solid, index=slice(-700, None))

In [ ]:
def compute_rdf(frames, cutoff, bins):
    """
    Compute the average Radial Distribution Function (RDF) over multiple frames.

    Parameters:
    - frames: List of ASE Atoms objects (MD trajectory snapshots)
    - cutoff: Maximum radius (in Å) for RDF calculation
    - bins: Number of histogram bins between 0 and cutoff

    Returns:
    - r: Array of radius values (bin centers)
    - avg_rdf: Averaged g(r) over all frames
    """
    all_rdfs = []

    for atoms in frames:
        # Filter only atoms of type 1 (assumes LAMMPS type info in atoms.arrays["type"])
        type_mask = atoms.arrays["type"] == 1
        positions = atoms.positions[type_mask]
        cell = atoms.get_cell()
        pbc = atoms.get_pbc()

        # Compute RDF for this frame
        rdf, r = radial_distribution_function(
            positions, positions, bins, cell, pbc, cutoff=cutoff
        )
        all_rdfs.append(rdf)

    # Average over all frames
    avg_rdf = np.mean(all_rdfs, axis=0)
    return r, avg_rdf


In [ ]:
# --------- Compute RDFs for both trajectories ---------
r_liq, g_liq = compute_rdf(frames_liquid, rdf_cutoff, rdf_bins)
r_sol, g_sol = compute_rdf(frames_solid, rdf_cutoff, rdf_bins)

In [ ]:
# --------- Plotting RDFs ---------
plt.figure(figsize=(8, 5))
plt.plot(r_liq, g_liq, label='Liquid', lw=2)
plt.plot(r_sol, g_sol, label='Solid', lw=2)
plt.xlabel('r (Å)', fontsize=12)
plt.ylabel('g(r)', fontsize=12)
plt.title('Radial Distribution Function (RDF)', fontsize=14)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("rdf_comparison.png")  # Save as image file
plt.show()